In [12]:
import os
import sys

import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pprint import pprint
from tqdm import tqdm
from astropy.io import fits

# set paths to various directories based on the machine this code is being executed on
try:
    with initialize(version_base=None, config_path='config'):  
        config = compose(config_name='config.yaml')  # overrides=['machine=uzay']
except:
    with initialize(version_base=None, config_path='../../config'):  
        config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import gs
from mejiro.lenses import lens_util

In [13]:
survey_params = util.hydra_to_dict(config.survey)
snr_band = survey_params['snr_band']

pipeline_params = util.hydra_to_dict(config.pipeline)
# debugging = pipeline_params['debugging']
debugging = False

if debugging:
    pipeline_dir = f'{config.machine.pipeline_dir}_dev'
else:
    pipeline_dir = config.machine.pipeline_dir

print(f'pipeline_dir: {pipeline_dir}')

pipeline_dir: /data/bwedig/mejiro/pipeline


In [14]:
all_lenses = lens_util.get_detectable_lenses(pipeline_dir, with_subhalos=True, verbose=True)

  0%|          | 27/10245 [00:00<00:38, 263.21it/s]

100%|██████████| 10245/10245 [03:20<00:00, 51.04it/s] 


In [15]:
import random

lens = random.choice(all_lenses)
uid = lens.uid
print(f'uid: {uid}')

fits_path = os.path.join(repo_dir, f'strong_lens_{str(uid).zfill(8)}.fits')
print(fits_path)

uid: 00000971
/nfshome/bwedig/mejiro/strong_lens_00000971.fits


In [16]:
arrays = []
glob_path = f'{pipeline_dir}/03/**/array_{str(uid).zfill(8)}_*.npy'
# print(glob_path)
array_paths = sorted(glob(glob_path))
# print(array_paths)

assert len(array_paths) == len(pipeline_params['bands']), 'Could not find an array for each band'

for array_path in array_paths:
    arrays.append(np.load(array_path))

In [17]:
images = []
glob_path = f'{pipeline_dir}/04/**/galsim_{str(uid).zfill(8)}_*.npy'
# print(glob_path)
image_paths = sorted(glob(glob_path))
# print(image_paths)

assert len(image_paths) == len(pipeline_params['bands']), 'Could not find an image for each band'

for image_path in image_paths:
    images.append(np.load(image_path))

In [18]:
pprint(lens.source_mags)
pprint(lens.lens_mags)

{'F087': 22.60419341029887,
 'F106': 21.98131471147577,
 'F129': 21.703222400018348,
 'F158': 21.430428296012046,
 'F184': 21.26342508765314}
{'F087': 23.039265684732985,
 'F106': 22.64781493981005,
 'F129': 22.325824958009736,
 'F158': 21.992020451324,
 'F184': 21.75233963291085}


In [19]:
# TODO TEMP
lens.detector = 'SCA01'
lens.detector_position = (2048, 2048)

In [26]:
primary_header = fits.Header()

# mejiro metadata
import mejiro
primary_header['VERSION'] = (mejiro.__version__, 'mejiro version')

import getpass
import platform
primary_header['AUTHOR'] = (f'{getpass.getuser()}@{platform.node()}', 'username@host for calculation')

from datetime import datetime
now_string = datetime.now().strftime("%Y_%m_%d-%I_%M_%S_%p")
primary_header['CREATED'] = (now_string, 'Date of calculation')

primary_header['UID'] = (lens.uid, 'UID for system assigned by mejiro')

# lens params
primary_header['ZSOURCE'] = (lens.z_source, 'Source galaxy redshift')
primary_header['ZLENS'] = (lens.z_lens, 'Lens galaxy redshift')
primary_header['DS'] = (lens.d_s, 'Comoving distance to source galaxy [Gpc]')
primary_header['DL'] = (lens.d_l, 'Comoving distance to lens galaxy [Gpc]')
primary_header['DLS'] = (lens.d_ls, 'Comoving distance between lens and source [Gpc]')
primary_header['HALOMASS'] = (lens.main_halo_mass, 'Lens galaxy main halo mass [M_sun]')
primary_header['THETAE'] = (lens.get_einstein_radius(), 'Einstein radius [arcsec]')
primary_header['SIGMAV'] = (lens.lens_vel_disp, 'Lens galaxy velocity dispersion [km/s]')
primary_header['MU'] = (lens.magnification, 'Flux-weighted magnification of source')
primary_header['NUMIMAGE'] = (lens.num_images, 'Number of images formed by the system')
primary_header['SNR'] = (lens.snr, f'Signal-to-noise ratio in {snr_band} band')

# subhalo params
primary_header['LOGMLOW'] = (pipeline_params['log_mlow'], 'Lower mass limit for subhalos [log10(M_sun)]')
primary_header['LOGMHIGH'] = (pipeline_params['log_mhigh'], 'Upper mass limit for subhalos [log10(M_sun)]')
primary_header['RTIDAL'] = (pipeline_params['r_tidal'], 'See pyHalo documentation')
primary_header['SIGMASUB'] = (pipeline_params['sigma_sub'], 'See pyHalo documentation')
primary_header['NUMSUB'] = (lens.num_subhalos, 'Number of subhalos')


primary_hdu = fits.PrimaryHDU(None, primary_header)
hdul = fits.HDUList([primary_hdu])

for band, image in zip(pipeline_params['bands'], images):
    header = fits.Header()
    header['INSTRUME'] = ('WFI', 'Instrument')
    header['FILTER'] = (band, 'Filter')
    header['EXPOSURE'] = (pipeline_params['exposure_time'], 'Exposure time [seconds]')
    header['OVERSAMP'] = (pipeline_params['grid_oversample'], 'Oversampling used in calculation')
    header['PIXELSCL'] = (0.11, 'Pixel scale [arcsec/pixel]')
    header['FOV'] = (0.11 * pipeline_params['final_pixel_side'], 'Field of view [arcsec]')
    header['DETECTOR'] = (lens.detector, 'Detector')
    header['DET_X'] = (lens.detector_position[0], 'Detector X position')
    header['DET_Y'] = (lens.detector_position[1], 'Detector Y position')

    # lens params
    header['SOURCMAG'] = (lens.source_mags[band], 'Unlensed source galaxy AB magnitude')
    header['MAGNMAG'] = (lens.lensed_source_mags[band], 'Lensed source galaxy AB magnitude')
    header['LENSMAG'] = (lens.lens_mags[band], 'Lens galaxy AB magnitude')

    image_hdu = fits.ImageHDU(image, header, name=band)
    hdul.append(image_hdu)

hdul.writeto(fits_path, overwrite=True)

In [27]:
fits.info(fits_path)

Filename: /nfshome/bwedig/mejiro/strong_lens_00000971.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      24   ()      
  1  F087          1 ImageHDU        20   (90, 90)   float32   
  2  F106          1 ImageHDU        20   (90, 90)   float32   
  3  F129          1 ImageHDU        20   (90, 90)   float32   
  4  F158          1 ImageHDU        20   (90, 90)   float32   
  5  F184          1 ImageHDU        20   (90, 90)   float32   


In [28]:
for i in range(4):
    pprint(fits.getheader(fits_path, ext=i))
    print('\n')

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
VERSION = '0.1     '           / mejiro version                                 
AUTHOR  = 'bwedig@rosemary.physics.wustl.edu' / username@host for calculation   
CREATED = '2024_10_21-04_51_04_PM' / Date of calculation                        
UID     = '00000971'           / UID for system assigned by mejiro              
ZSOURCE =   1.4021943913609833 / Source galaxy redshift                         
ZLENS   =   0.7308084499279041 / Lens galaxy redshift                           
DS      =     4.28843628908031 / Comoving distance to source galaxy [Gpc]       
DL      =   2.6700071480133047 / Comoving distance to lens galaxy [Gpc]         
DLS     =    1.6184291410670